# 08 0xA

### メモ  
GAOZで完全一致しないやつがある  
->ハミング距離最小のやつで絞る



### 担当者と日付
2024/10/1 渡邉健  
2024/10/3 渡邉

## 準備

### ライブラリのインポート  

In [1]:
import pandas as pd
import numpy as np
import warnings #ヘッダとかのやつが邪魔だったのでファイルにまとめてインポートしてる
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
# fmt: off
ALL_HEADER = ['Name', 'Gender', 'Age', 'Occupation', 'ZIP-code', '2', '56', '247', '260', '653', '673', '810', '885',  # noqa: E501
                      '1009', '1073', '1097', '1126', '1525', '1654', '1702', '1750', '1881', '1920', '1967', '2017',  # noqa: E501
                      '2021', '2043', '2086', '2087', '2093', '2100', '2105', '2138', '2143', '2174', '2193', '2253',  # noqa: E501
                      '2399', '2628', '2797', '2872', '2968', '3393', '3438', '3439', '3440', '3466', '3479', '3489',  # noqa: E501
                      '3877', '3889'] # noqa

B_HEADERS_LIST = [
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '260', '653', '1525', '2105', '2193', '2253', '2628', '2872', '3438', '3439', '3440', '3877', '3889'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '2', '56', '260', '653', '673', '1009', '1073', '1525', '1750', '1881', '1967', '2043', '2093', '2105', '2143', '2193', '2399', '2628', '2968', '3479', '3489', '3877', '3889'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '673', '1881', '1920', '2087', '2138'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '2', '56', '673', '810', '885', '1009', '1073', '1097', '1525', '1654', '1702', '1750', '1881', '1920', '1967', '2017', '2043', '2087', '2093', '2138', '2399', '3438', '3439', '3440'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '673', '810', '1073', '1126', '1702', '2100', '2174', '2253', '2797', '3393', '3466'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '247', '885', '1097', '1654', '2086', '2138', '2872'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '247', '2100', '2143', '2872', '3479'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '260', '1097', '1750', '2021', '2093', '2105', '2628', '2968'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '247', '1920', '2017', '2087'],  # noqa: E501
  ['Gender', 'Age', 'Occupation', 'ZIP-code', '260', '1097', '2628', '2174', '2797', '1073', '2100', '2968', '2105', '2193'],  # noqa: E501
]

B_USER_ATTRIBUTE_HEADERS = ['Gender', 'Age', 'Occupation', 'ZIP-code']

B_REVIEW_HEADERS_LIST = [
  ['260', '653', '1525', '2105', '2193', '2253', '2628', '2872', '3438', '3439', '3440', '3877', '3889'],  # noqa: E501
  ['2', '56', '260', '653', '673', '1009', '1073', '1525', '1750', '1881', '1967', '2043', '2093', '2105', '2143', '2193', '2399', '2628', '2968', '3479', '3489', '3877', '3889'],  # noqa: E501
  ['673', '1881', '1920', '2087', '2138'],  # noqa: E501
  ['2', '56', '673', '810', '885', '1009', '1073', '1097', '1525', '1654', '1702', '1750', '1881', '1920', '1967', '2017', '2043', '2087', '2093', '2138', '2399', '3438', '3439', '3440'],  # noqa: E501
  ['673', '810', '1073', '1126', '1702', '2100', '2174', '2253', '2797', '3393', '3466'],  # noqa: E501
  ['247', '885', '1097', '1654', '2086', '2138', '2872'],  # noqa: E501
  ['247', '2100', '2143', '2872', '3479'],  # noqa: E501
  ['260', '1097', '1750', '2021', '2093', '2105', '2628', '2968'],  # noqa: E501
  ['247', '1920', '2017', '2087'],  # noqa: E501
  ['260', '1097', '2628', '2174', '2797', '1073', '2100', '2968', '2105', '2193'],  # noqa: E501
]

MOVIE_IDS = ['2', '56', '247', '260', '653', '673', '810', '885', '1009', '1073', '1097', '1126', '1525', '1654', '1702', '1750', '1881', '1920', '1967', '2017', '2021', '2043', '2086', '2087', '2093', '2100', '2105', '2138', '2143', '2174', '2193', '2253', '2399', '2628', '2797', '2872', '2968', '3393', '3438', '3439', '3440', '3466', '3479', '3489', '3877', '3889']
# fmt: on

### C_0 ~ C_9までの読み込み

In [3]:
from os import listdir

ATTACK_TARGET = "21"

#c0~c9に対するdfのリスト
c_data_list = []

files = listdir("../data")

for i in range(10):
    file = None
    for f in files:
        if f.startswith(f"C{ATTACK_TARGET}_{i}"):
            file = f
            break
    if file is None:
        print(f)
        raise("File not found")
    c_data = pd.read_csv(f"../data/C{ATTACK_TARGET}_{i}.csv")
    c_data_list.append(c_data)

c_data_list[0]

,Gender,Age,Occupation,ZIP-code,260,653,1525,2105,2193,2253,2628,2872,3438,3439,3440,3877,3889
0,F,18,12,136,0,0,0,0,0,0,4,0,2,0,0,0,0
1,F,25,17,967,0,4,3,2,0,0,4,4,1,0,1,0,0
2,F,18,6,445,0,0,0,4,1,0,2,0,0,2,3,4,0
3,M,50,5,523,0,0,0,5,2,1,2,2,3,0,0,0,5
4,M,1,17,321,3,4,4,3,4,1,0,3,3,5,0,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,F,25,4,752,2,1,0,5,0,2,4,0,0,1,3,0,0
9996,M,18,13,532,3,2,0,5,2,4,0,0,1,0,1,5,3
9997,M,35,6,210,2,1,0,4,0,2,5,5,2,2,0,0,3
9998,M,1,12,494,4,2,4,5,1,3,0,1,1,3,3,5,3


### Ba, Bbの読み込み

In [4]:
Ba = pd.read_csv(f"../data/B{ATTACK_TARGET}a.csv", names=ALL_HEADER)
Ba = Ba.drop(0)
Ba.head()

,Name,Gender,Age,Occupation,ZIP-code,2,56,247,260,653,...,2968,3393,3438,3439,3440,3466,3479,3489,3877,3889
1,Yank Schiell,M,50,16,104,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Prince Tibbotts,M,1,4,132,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Martainn Mellem,M,50,12,971,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Dorey Pittson,M,50,0,954,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Dav Wakeling,M,50,10,146,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
Bb = pd.read_csv(f"../data/B{ATTACK_TARGET}b.csv")
Bb.head()

,2,56,247,260,653,673,810,885,1009,1073,...,2968,3393,3438,3439,3440,3466,3479,3489,3877,3889
0,0,0,3,1,2,5,3,0,0,*,...,1,0,4,3,0,0,0,2,0,4
1,5,3,4,4,0,0,0,0,5,1,...,0,0,0,0,0,0,4,2,0,0
2,0,1,5,2,0,3,0,0,0,1,...,0,3,2,0,0,0,4,1,0,1
3,4,3,4,1,0,2,0,3,*,4,...,1,4,5,1,2,4,5,0,0,4
4,0,4,5,2,5,5,0,4,0,2,...,0,0,2,0,0,0,3,3,0,1


## 分析

### GAOZに関して、BaのN番目の行と完全一致する行をC0~C9から取得　　
-GAOZへの変更が行われているか

In [18]:
# 試しにBaの3行目を選択
N = 4

gender = Ba.iloc[N]["Gender"]
age = int(Ba.iloc[N]["Age"])
occupation = int(Ba.iloc[N]["Occupation"])
zipcode = int(Ba.iloc[N]["ZIP-code"])

print("検索条件")
print(gender, age, occupation, zipcode)

検索条件
M 50 10 146


### GAOZが完全一致するものでなく、各Ciの中でHamming distが最小のものをリスト化する

In [19]:
import copy

def GAOZ_weighted_hamming(comp, i_gender, i_age, i_occupation, i_zipcode):
    ham = 0
    if comp['Gender'] != i_gender:
        ham += 1
    if comp['Age'] != i_age:
        ham += 2
    if  comp['Occupation'] != i_occupation:
        ham += 2
    if comp['ZIP-code'] != i_zipcode:
        ham += 3
    return ham

filtered_C_list = []
for c_index, cf in enumerate(c_data_list):
    min_hamming_distance = float("inf")
    cand = []
    # Cの10000レコードのそれぞれにハミング距離を計算
    for index, c_row in cf.iterrows():
        hamming_distance = GAOZ_weighted_hamming(c_row, gender, age, occupation, zipcode)
        if hamming_distance == min_hamming_distance:
            cand.append((c_row, hamming_distance))
        if hamming_distance < min_hamming_distance:
            min_hamming_distance = hamming_distance
            cand = [(c_row, hamming_distance)]
    # ハミング距離とCのファイルのindexを追加
    for elem, hamming_distance in cand:
        headers = copy.deepcopy(ALL_HEADER)
        headers.append("GAOZ_Hamming_Dist")
        headers.append("C_index")
        elem["GAOZ_Hamming_Dist"] = hamming_distance
        elem["C_index"] = c_index
        filtered_C_list.append(pd.Series(elem, index=headers))

merged_df = pd.DataFrame(filtered_C_list)
merged_df

,Name,Gender,Age,Occupation,ZIP-code,2,56,247,260,653,...,3438,3439,3440,3466,3479,3489,3877,3889,GAOZ_Hamming_Dist,C_index
2606,NaN,M,50,4,146,NaN,NaN,NaN,2.0,0.0,...,5.0,0.0,2.0,NaN,NaN,NaN,0.0,0.0,2,0
374,NaN,M,50,11,146,0.0,3.0,NaN,4.0,1.0,...,NaN,NaN,NaN,NaN,2.0,0.0,4.0,1.0,2,1
8702,NaN,M,50,13,146,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2
2015,NaN,M,50,19,146,1.0,4.0,NaN,NaN,NaN,...,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,2,3
5010,NaN,M,50,0,146,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,2,4
4114,NaN,F,50,10,146,NaN,NaN,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,5
9477,NaN,M,50,14,146,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,2,6
407,NaN,M,50,8,146,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,7
9595,NaN,M,50,6,146,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,8
58,NaN,M,50,17,146,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,9


In [20]:
# GAOZ_Hamming_Distが小さい順に並べ替える
sorted_df = merged_df.sort_values(by="GAOZ_Hamming_Dist").reset_index(drop=True)

# 各GAOZ_Hamming_Distのグループごとにランダムシャッフル
grouped_df = sorted_df.groupby("GAOZ_Hamming_Dist", group_keys=False).apply(lambda x: x.sample(frac=1).reset_index(drop=True))

# 各カラムごとに最初の非NaN値を選択する関数
def select_first_non_nan(series):
    return series.dropna().iloc[0] if not series.dropna().empty else None

# 各カラムから値を選択
selected_values = {col: select_first_non_nan(grouped_df[col]) for col in grouped_df.columns}

# 結果を表示
selected_values_df = pd.DataFrame([selected_values])
selected_values_df

/var/folders/22/sn7bjw057h54_810zqj94q680000gn/T/ipykernel_30311/647578683.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_df = sorted_df.groupby("GAOZ_Hamming_Dist", group_keys=False).apply(lambda x: x.sample(frac=1).reset_index(drop=True))


,Name,Gender,Age,Occupation,ZIP-code,2,56,247,260,653,...,3438,3439,3440,3466,3479,3489,3877,3889,GAOZ_Hamming_Dist,C_index
0,None,F,50,10,146,0.0,3.0,2.0,2.0,0.0,...,5.0,0.0,2.0,2.0,2.0,0.0,0.0,0.0,1,5


### データの再現　　
各行に対して最頻値を一つ値を取得する

In [12]:
import statistics
from collections import Counter

def select_random_mode(column):
    """
    最頻値を取得
    最頻値が複数ある場合はランダムに1つ選択
    """
    # NaNを除外した値のリストを作成
    valid_values = column.dropna().tolist()
    if not valid_values:
        # 有効な値がない場合はNaNを返す
        return np.nan
    # 最頻値とその出現回数を取得
    value_counts = Counter(valid_values)
    max_count = max(value_counts.values())
    # 最頻値のリストを作成
    modes = [value for value, count in value_counts.items() if count == max_count]
    # 最頻値が1つの場合はそれを返し、複数ある場合はランダムに1つ選択
    return np.random.choice(modes)

def select_random_value(column):
    # NaNを除外した値のリストを作成
    valid_values = column.dropna().tolist()
    if valid_values:
        # 有効な値がある場合、ランダムに1つ選択
        return np.random.choice(valid_values)
    else:
        # 有効な値がない場合はNaNを返す
        return np.nan

# 各カラムからランダムに値を選択
# random_values = {}
# for column in ALL_HEADER:
#     random_values[column] = select_random_value(merged_df[column])

# # 結果を新しいデータフレームとして作成
# result_df = pd.DataFrame([random_values])
# result_df

## 攻撃

### 以上の処理をBaの各行（0~49行）に対して実行する

In [21]:
import pandas as pd
import numpy as np

final_df = pd.DataFrame(columns=ALL_HEADER)

for row_index in range(50):
    # Baデータフレームから現在の行の情報を取得
    gender = Ba.iloc[row_index]["Gender"]
    age = int(Ba.iloc[row_index]["Age"])
    occupation = int(Ba.iloc[row_index]["Occupation"])
    zipcode = int(Ba.iloc[row_index]["ZIP-code"])

    filtered_C_list = []
    for c_index, cf in enumerate(c_data_list):
        min_hamming_distance = float("inf")
        cand = []
        # Cの10000レコードのそれぞれにハミング距離を計算
        for index, c_row in cf.iterrows():
            hamming_distance = GAOZ_weighted_hamming(c_row, gender, age, occupation, zipcode)
            if hamming_distance == min_hamming_distance:
                cand.append((c_row, hamming_distance))
            if hamming_distance < min_hamming_distance:
                min_hamming_distance = hamming_distance
                cand = [(c_row, hamming_distance)]
        # ハミング距離とCのファイルのindexを追加
        for elem, hamming_distance in cand:
            headers = copy.deepcopy(ALL_HEADER)
            headers.append("GAOZ_Hamming_Dist")
            headers.append("C_index")
            elem["GAOZ_Hamming_Dist"] = hamming_distance
            elem["C_index"] = c_index
            filtered_C_list.append(pd.Series(elem, index=headers))

    merged_df = pd.DataFrame(filtered_C_list)

    candidate_rows_df = pd.DataFrame(filtered_C_list)

    # GAOZ_Hamming_Distが小さい順に並べ替える
    sorted_df = candidate_rows_df.sort_values(by="GAOZ_Hamming_Dist").reset_index(drop=True)

    # 各GAOZ_Hamming_Distのグループごとにランダムシャッフル
    grouped_df = sorted_df.groupby("GAOZ_Hamming_Dist", group_keys=False).apply(lambda x: x.sample(frac=1).reset_index(drop=True))

    # 各カラムごとに最初の非NaN値を選択する関数
    def select_first_non_nan(series):
        return series.dropna().iloc[0] if not series.dropna().empty else None

    # 各カラムから値を選択
    selected_values = {col: select_first_non_nan(grouped_df[col]) for col in grouped_df.columns}


    # 各列に対して、ランダムに値を選択（多数決の実装）
    elected_value = {}
    for column in ALL_HEADER:
        elected_value[column] = selected_values[column]

    # 選択された値から新しい行を作成
    elected_value_row_df = pd.DataFrame([elected_value])

    # 選択された行をfinal_dfの最後に追加
    final_df = pd.concat([final_df, elected_value_row_df], ignore_index=True)

final_df

/var/folders/22/sn7bjw057h54_810zqj94q680000gn/T/ipykernel_30311/122308287.py:42: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_df = sorted_df.groupby("GAOZ_Hamming_Dist", group_keys=False).apply(lambda x: x.sample(frac=1).reset_index(drop=True))
/var/folders/22/sn7bjw057h54_810zqj94q680000gn/T/ipykernel_30311/122308287.py:42: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_df = sorted_df.grou

,Name,Gender,Age,Occupation,ZIP-code,2,56,247,260,653,...,2968,3393,3438,3439,3440,3466,3479,3489,3877,3889
0,None,M,50,4,104,2.0,3.0,4.0,1.0,0.0,...,0.0,0.0,3.0,2.0,0.0,0.0,0.0,5.0,5.0,0.0
1,None,M,1,4,132,0.0,1.0,4.0,0.0,1.0,...,3.0,1.0,5.0,0.0,0.0,5.0,3.0,2.0,3.0,1.0
2,None,F,50,12,971,5.0,0.0,1.0,1.0,4.0,...,4.0,0.0,1.0,3.0,0.0,0.0,5.0,1.0,1.0,4.0
3,None,M,50,0,954,4.0,1.0,1.0,2.0,0.0,...,0.0,0.0,2.0,0.0,0.0,0.0,5.0,3.0,0.0,5.0
4,None,F,50,10,146,1.0,4.0,2.0,2.0,0.0,...,0.0,0.0,1.0,1.0,0.0,2.0,2.0,0.0,0.0,0.0
5,None,F,50,12,750,5.0,4.0,5.0,1.0,3.0,...,2.0,0.0,1.0,1.0,0.0,1.0,4.0,0.0,5.0,0.0
6,None,F,1,10,919,5.0,1.0,4.0,3.0,0.0,...,4.0,3.0,3.0,0.0,0.0,3.0,4.0,0.0,0.0,0.0
7,None,F,25,18,26,2.0,1.0,4.0,5.0,1.0,...,0.0,1.0,2.0,0.0,4.0,2.0,0.0,0.0,0.0,4.0
8,None,M,50,13,331,5.0,0.0,2.0,0.0,3.0,...,0.0,5.0,5.0,0.0,3.0,0.0,2.0,1.0,0.0,0.0
9,None,F,45,8,23,4.0,1.0,3.0,2.0,2.0,...,0.0,5.0,5.0,2.0,0.0,0.0,2.0,5.0,1.0,0.0


### 最終的に得た映画のレビューの候補とBbを比較し、Bbの何番目のindexが一番近くてハミング距離にどれくらいの違いがあるかを取得する

In [22]:

def hamming_distance(a, b):
    a_int = a.fillna(0).astype(int)
    b_int = b.replace('*', None).fillna(0).astype(int)
    # ハミング距離の計算
    return np.sum(a_int != b_int)

def find_closest_row(row, Bb):
    distances = Bb.apply(lambda bb_row: hamming_distance(row[MOVIE_IDS], bb_row[MOVIE_IDS]), axis=1)
    min_distance = distances.min()
    closest_index = distances.idxmin()
    return pd.Series({'Min_Hamming_Distance': min_distance, 'Closest_Bb_Index': closest_index})

# 結果を格納するデータフレーム
result_df = pd.DataFrame(index=final_df.index, columns=['Min_Hamming_Distance', 'Closest_Bb_Index'])

# 各行に対してハミング距離を計算し、最も近い行を見つける
for idx, row in final_df[MOVIE_IDS].iterrows():
    result = find_closest_row(row, Bb[MOVIE_IDS])
    result_df.loc[idx] = result

# 元のfinal_dfにハミング距離の結果を結合
final_result_df = pd.concat([final_df, result_df], axis=1)
final_result_df

,Name,Gender,Age,Occupation,ZIP-code,2,56,247,260,653,...,3438,3439,3440,3466,3479,3489,3877,3889,Min_Hamming_Distance,Closest_Bb_Index
0,None,M,50,4,104,2.0,3.0,4.0,1.0,0.0,...,3.0,2.0,0.0,0.0,0.0,5.0,5.0,0.0,26,2
1,None,M,1,4,132,0.0,1.0,4.0,0.0,1.0,...,5.0,0.0,0.0,5.0,3.0,2.0,3.0,1.0,26,24
2,None,F,50,12,971,5.0,0.0,1.0,1.0,4.0,...,1.0,3.0,0.0,0.0,5.0,1.0,1.0,4.0,26,24
3,None,M,50,0,954,4.0,1.0,1.0,2.0,0.0,...,2.0,0.0,0.0,0.0,5.0,3.0,0.0,5.0,23,7
4,None,F,50,10,146,1.0,4.0,2.0,2.0,0.0,...,1.0,1.0,0.0,2.0,2.0,0.0,0.0,0.0,26,7
5,None,F,50,12,750,5.0,4.0,5.0,1.0,3.0,...,1.0,1.0,0.0,1.0,4.0,0.0,5.0,0.0,27,43
6,None,F,1,10,919,5.0,1.0,4.0,3.0,0.0,...,3.0,0.0,0.0,3.0,4.0,0.0,0.0,0.0,25,48
7,None,F,25,18,26,2.0,1.0,4.0,5.0,1.0,...,2.0,0.0,4.0,2.0,0.0,0.0,0.0,4.0,26,34
8,None,M,50,13,331,5.0,0.0,2.0,0.0,3.0,...,5.0,0.0,3.0,0.0,2.0,1.0,0.0,0.0,29,17
9,None,F,45,8,23,4.0,1.0,3.0,2.0,2.0,...,5.0,2.0,0.0,0.0,2.0,5.0,1.0,0.0,28,31
